In [1]:
import torch
from torch import nn
from torchvision import models
import os
from PIL import Image
import numpy as np
from tqdm import tqdm
from numpy.random import shuffle
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from torchvision import transforms
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import cv2
import torchvision.transforms as T
import random
import pandas as pd

In [2]:
def set_all_seeds(seed=42):
    # python's seeds
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    
    # torch's seeds
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
set_all_seeds()

In [3]:
device = 'cuda'

In [4]:
device

'cuda'

In [5]:
size = (128, 128)

In [6]:
transform = T.Compose([
    # Случайное изменение яркости, контрастности и насыщенности
    T.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
    
    # Случайный поворот изображения
    T.RandomRotation(degrees=15),
    
    # Случайное горизонтальное отражение
    T.RandomHorizontalFlip(p=0.5),
    
    # Случайное вертикальное отражение
    T.RandomVerticalFlip(p=0.2),
    
    # Случайное изменение перспективы
    T.RandomPerspective(distortion_scale=0.5, p=0.5),
    
    # Случайное масштабирование и обрезка
    T.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0)),
    
    # Случайное обрезание части изображения
    T.RandomCrop(size=(200, 200)),
    
    # Преобразование в тензор и нормализация
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [24]:
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.RandomHorizontalFlip(0.3),
                transforms.RandomVerticalFlip(0.3),
                transforms.RandomRotation(20),
                transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 3)),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])

In [25]:
classes = os.listdir(r'journey-springfield\train\simpsons_dataset')
dct_to = {classes[i]: i for i in range(len(classes))}

In [26]:
transf = transforms.Compose([transforms.ToTensor(), transforms.Normalize([0.5], [0.5])])

In [27]:
X = []
y = []
for cl in tqdm(classes):
    pth = os.path.join('journey-springfield', 'train', 'simpsons_dataset', cl)
    for img in os.listdir(pth):
        img = cv2.imread(os.path.join(pth, img))
        img = Image.fromarray(img)
        img = img.resize(size)
        img = transform(img)
        X.append(img)
        y.append(dct_to[cl])

100%|██████████| 42/42 [02:01<00:00,  2.89s/it]


In [28]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=22, shuffle=True)

In [29]:
train_dataset = [(X_train[i], y_train[i]) for i in range(len(X_train))]
test_dataset = [(X_test[i], y_test[i]) for i in range(len(X_test))]

In [53]:
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size, drop_last=True)
test_loader = DataLoader(test_dataset, batch_size, drop_last=True)

In [54]:
class Metric:
    def __init__(self, func, **kwargs):
        self.func = func
        self.arg = kwargs
    
    def __call__(self, y_true, y_pred):
        return self.func(y_true, y_pred, **self.arg)
    
    def __name__(self):
        return self.func.__name__

In [55]:
def train(model, optim, loss_func, loader, epochs, device):
    fin = []
    model.to(device)  # Переносим модель на устройство
    model.train()
    for u in range(1, epochs + 1):
        print(f'Epoch: {u}')
        hist = []
        for X, y in tqdm(loader):
            X, y = X.to(device, dtype=torch.float), y.to(device)  # Переносим данные на устройство
            pred = model(X)
            loss = loss_func(pred, y)
            hist.append(loss.item())
            loss.backward()
            optim.step()
            optim.zero_grad()
        print(f'Loss: {sum(hist) / len(hist)}')
        fin.append(sum(hist) / len(hist))
    return fin


In [56]:
def test_model(model, loader, metrics, device):
    hist_pred = []
    hist_true = []

    model.to(device)  # Переносим модель на устройство
    model.eval()
    with torch.no_grad():  # Отключаем вычисление градиентов
        for X, y in tqdm(loader):
            X, y = X.to(device, dtype=torch.float), y.to(device)  # Переносим данные на устройство
            pred = model(X)
            hist_pred.extend(list(np.argmax(pred.cpu().detach().numpy(), axis=1)))  # Переносим на CPU перед numpy
            hist_true.extend(list(y.cpu().detach().numpy()))

    for x in metrics:
        yield (f'{x.__name__()}', x(hist_true, hist_pred))  # Исправлено на x.__name__ вместо x.__name__()



In [75]:
model = models.resnet18(pretrained=True)

In [76]:
model.fc = nn.Sequential(
    nn.Linear(512, 256),
    nn.BatchNorm1d(256),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(256, 128),
    nn.BatchNorm1d(128),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(128, 64),
    nn.BatchNorm1d(64),
    nn.Dropout(),
    nn.ReLU(),
    nn.Linear(64, 42)
)

In [77]:
model = model.to(device)

In [78]:
opt = torch.optim.RMSprop(model.parameters(), 3e-4)
loss = nn.CrossEntropyLoss()

In [80]:
train(model, opt, loss, train_loader, 10, device)

Epoch: 1


 32%|███▏      | 145/457 [11:25<24:34,  4.73s/it]  


KeyboardInterrupt: 

In [ ]:
alexnet_rez = test_model(model, test_loader, [Metric(accuracy_score), Metric(precision_score, average='weighted'),  Metric(recall_score, average='weighted'),  Metric(f1_score, average='weighted')], device)
alexnet_rez = list(alexnet_rez)

100%|██████████| 196/196 [00:07<00:00, 27.25it/s]
C:\Users\User\AppData\Roaming\Python\Python311\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
alexnet_rez

[('accuracy_score', 0.8163265306122449),
 ('precision_score', 0.8299957496719652),
 ('recall_score', 0.8163265306122449),
 ('f1_score', 0.8140024211046469)]

In [ ]:
def predict_images(model, test_dir, transf, size, device):
    preds = []
    model.to(device)  # Переносим модель на устройство
    model.eval()
    
    with torch.no_grad():  # Отключаем вычисление градиентов
        for x in tqdm(os.listdir(test_dir)):
            # Путь к изображению
            pth = os.path.join(test_dir, x)
            img = cv2.imread(pth)
            img = Image.fromarray(img)
            img = img.resize(size)
            img = transf(img)
            img = img.unsqueeze(0).to(device)  # Добавляем размерность батча и переносим на устройство
            
            # Прогноз
            pred = model(img)
            pred = np.argmax(pred.cpu().detach().numpy(), axis=1)[0]  # Переносим на CPU перед numpy
            preds.append([x, pred])
    
    return preds

In [ ]:
test_dir = os.path.join('journey-springfield', 'testset', 'testset')
size = (128, 128)  # Задаем размер изображения
predictions = predict_images(model, test_dir, transf, size, device)

100%|██████████| 991/991 [00:07<00:00, 139.08it/s]


In [ ]:
predictions

[['img0.jpg', 29],
 ['img1.jpg', 4],
 ['img10.jpg', 28],
 ['img100.jpg', 7],
 ['img101.jpg', 2],
 ['img102.jpg', 16],
 ['img103.jpg', 11],
 ['img104.jpg', 7],
 ['img105.jpg', 20],
 ['img106.jpg', 16],
 ['img107.jpg', 18],
 ['img108.jpg', 22],
 ['img109.jpg', 16],
 ['img11.jpg', 28],
 ['img110.jpg', 27],
 ['img111.jpg', 17],
 ['img112.jpg', 0],
 ['img113.jpg', 11],
 ['img114.jpg', 16],
 ['img115.jpg', 0],
 ['img116.jpg', 22],
 ['img117.jpg', 29],
 ['img118.jpg', 9],
 ['img119.jpg', 24],
 ['img12.jpg', 15],
 ['img120.jpg', 2],
 ['img121.jpg', 17],
 ['img122.jpg', 17],
 ['img123.jpg', 28],
 ['img124.jpg', 11],
 ['img125.jpg', 2],
 ['img126.jpg', 7],
 ['img127.jpg', 2],
 ['img128.jpg', 28],
 ['img129.jpg', 6],
 ['img13.jpg', 0],
 ['img130.jpg', 15],
 ['img131.jpg', 37],
 ['img132.jpg', 20],
 ['img133.jpg', 6],
 ['img134.jpg', 7],
 ['img135.jpg', 16],
 ['img136.jpg', 0],
 ['img137.jpg', 25],
 ['img138.jpg', 28],
 ['img139.jpg', 0],
 ['img14.jpg', 11],
 ['img140.jpg', 9],
 ['img141.jpg', 7],

In [ ]:
dct_out = {y: x for x, y in dct_to.items()}

In [ ]:
data = pd.DataFrame(data=predictions, columns=['Id', 'Expected'])
data['Expected'] = data['Expected'].map(dct_out)

In [ ]:
data

,Id,Expected
0,img0.jpg,nelson_muntz
1,img1.jpg,bart_simpson
2,img10.jpg,ned_flanders
3,img100.jpg,chief_wiggum
4,img101.jpg,apu_nahasapeemapetilon
...,...,...
986,img987.jpg,edna_krabappel
987,img988.jpg,homer_simpson
988,img989.jpg,charles_montgomery_burns
989,img99.jpg,chief_wiggum


In [ ]:
data.to_csv(r'answer.csv', index=False)